In [3]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import imp
import help_tools as ht
HTML("<style>.container { width: 120% !important; }</style>")

In [ ]:
api_key_tmdb = '' 
api_key_omdb = ''

In [41]:
# откуда накидывать id людей
# https://www.themoviedb.org/person/7030-henry-goodman

actor_dict = {
            'joaquin-phoenix' : 73421,
            'colin-farrell' : 72466,
            'leonardo-dicaprio' : 6193,
            'javier-bardem' : 3810,
            'anthony-hopkins' : 4173,
            'brad-pitt' : 287,
            'antonio-banderas': 3131,
            'matthew-mcconaughey' : 10297,
            'steve-buscemi' : 884,
            'tim-roth' : 3129,
            'mads-mikkelsen' : 1019,
            'scarlett-johansson' : 1245,
            'til-schweiger' : 1844
             }

director_dict = {
#                 'quentin-tarantino' : 138,
#                 'david-fincher' : 7467,
#                 'jim-jarmusch' : 4429,
#                 'guy-ritchie' : 956,
#                 'christopher-nolan' : 525,
#                 'darren-aronofsky' : 6431,
#                 'martin-mcdonagh' : 54472,
#                 'pedro-almodovar' : 309,
#                 'paolo-sorrentino' : 56194,
#                 'cristian-mungiu' : 20657,
#                 'yorgos-lanthimos' : 122423,
#                 'bong-joon-ho' : 21684,
#                 'lars-von-trier' : 42
}

# фильмы которые не хочется чекать (уже просмотренные или не интересные)
excl_films = [335977, 799583, 882569, 800158, 872585, 696506, 1029955]

# предварительные фильтры (по имеющимся параметрам в фильмографии - чтобы дальше не тратить запросы на детализацию)
vote_average_min = 5
vote_count_min = 100
release_date_min = '2022-01-01'
language = 'ru' # добавлять русскояз названия если есть в прокате 

In [42]:
# ПОЛУЧАЕМ ИНФУ ИЗ TMDB + OMDB (РАБОТА С API)
flt_dict = {'vote_average_min' : vote_average_min, 
            'vote_count_min' : vote_count_min,
            'release_date_min' : release_date_min,
            'language' : language
           }

# получаем инфу по списку актеров
df1 = ht.get_actors_filmo(api_key_tmdb, actor_dict, excl_films = excl_films, flt_dict = flt_dict)
# получаем инфу по списку режиссеров
df2 = ht.get_directors_filmo(api_key_tmdb, director_dict, excl_films = np.append(df1.movie_id.values, excl_films), flt_dict = flt_dict)
# итого
df = pd.concat([df1, df2], ignore_index = True)
df_final = ht.get_imdb_info(api_key_omdb, df)

In [46]:
# ПОСТ ФИЛЬТРАЦИЯ И СОРТИРОВКИ ИЗ ДОП УСЛОВИЙ (НЕТ РАБОТЫ С API)
post_excl_films = []
ht.get_filter(df_final, 
           vote_count_min=100, # vote cnt on tmdb
           runtime_min = 70, # film duration
           vote_average_min = 5, # rating по tmdb
           imdb_rating_min = 6, 
           release_date_min = '2023-01-01',
           genres_flt = ['Animation', 'Documentary'], # жанры которые неинтересно отображать
           excl_films = np.append(excl_films, post_excl_films), # фильмы которые неинтересно отображать
           drop_columns=['movie_status', 'movie_id', 'countries'] # какие колонки скрыть
          ).sort_values(by='imdb_rating', ascending=False)

,person,person_type,title,original_title,directors,release_date,imdb_rating,awards,genres,runtime,imdb_id,vote_average,vote_count
7,javier-bardem,actor,Дюна: Часть вторая,Dune: Part Two,Denis Villeneuve,2024-02-27,8.5,Won 2 Oscars. 106 wins & 367 nominations total,"['Science Fiction', 'Adventure']",167,tt15239678,8.147,6570
3,mads-mikkelsen,actor,Меч короля,Bastarden,Nikolaj Arcel,2023-10-05,7.7,22 wins & 17 nominations total,"['Action', 'Drama', 'History']",127,tt20561198,7.700,402
2,matthew-mcconaughey,actor,Дэдпул и Росомаха,Deadpool & Wolverine,Shawn Levy,2024-07-24,7.6,18 wins & 53 nominations total,"['Action', 'Comedy', 'Science Fiction']",128,tt6263850,7.603,7104
4,leonardo-dicaprio,actor,Убийцы цветочной луны,Killers of the Flower Moon,Martin Scorsese,2023-10-18,7.6,Nominated for 10 Oscars. 136 wins & 427 nomina...,"['Crime', 'History', 'Drama']",206,tt5537002,7.430,3576
12,anthony-hopkins,actor,Одна жизнь,One Life,James Hawes,2023-12-21,7.5,3 wins total,"['History', 'Drama']",109,tt13097932,7.802,794
8,javier-bardem,actor,Русалочка,The Little Mermaid,Rob Marshall,2023-05-18,7.2,3 wins & 33 nominations total,"['Adventure', 'Family', 'Fantasy', 'Romance']",135,tt5971474,6.316,3043
6,joaquin-phoenix,actor,Все страхи Бо,Beau Is Afraid,Ari Aster,2023-04-14,6.7,32 nominations total,"['Comedy', 'Adventure', 'Fantasy']",179,tt13521006,6.740,1105
10,antonio-banderas,actor,Приключения Паддингтона 3,Paddington in Peru,Dougal Wilson,2024-11-08,6.7,2 wins & 5 nominations total,"['Family', 'Comedy', 'Adventure']",106,tt5822536,6.838,349
0,scarlett-johansson,actor,Покажи мне Луну,Fly Me to the Moon,Greg Berlanti,2024-07-10,6.6,2 wins & 3 nominations total,"['Romance', 'Comedy']",132,tt1896747,6.839,912
1,scarlett-johansson,actor,Город астероидов,Asteroid City,Wes Anderson,2023-06-08,6.5,5 wins & 90 nominations total,"['Comedy', 'Drama']",105,tt14230388,6.461,2072


### Spotify

In [29]:
# get id + secret on your : https://developer.spotify.com/dashboard/e7bd2deea29b4fe1a33392499c720f45
client_id = ''
client_secret = ''

In [38]:
# посмотреть айдишки наиболее популярных исполнителей по имени
artist_name = 'linkin'
top_cnt = 5
ht.get_top_n_id_by_name(client_id, client_secret, artist_name, top_cnt)

['Linkin Park - 6XyY86QOPPrYVGvF9ch6wz - popularity: 90',
 'LinkingHearts - 69JI4UxwlMCDaDetq9EkVh - popularity: 12',
 'Linkin Leigh - 5Z3dLd7QFqMz7OnEqzFx8S - popularity: 13',
 'Professor Linkin - 6V2AiuoySMqgIruIa1SiQZ - popularity: 12',
 'DJ Linkinn - 34b64eKrxc3D9ZMKXF57iE - popularity: 6']

In [39]:
artist_list = [
#                 '132sZpCaM8ie6byAEcOcRs', # Haelos
#                '2VYQTNDsvvKN9wmU5W7xpj', # manson
#                 '7Ln80lUS6He07XvHI8qqHH', # arctic monk
#                 '2NPduAUeLVsfIauhRwuft1', # nightwish
#                 '5nPOO9iTcrs9k6yFffPxjH', # Röyksopp
#                 '2a5G7JLmVJNjfFNg8rwLcP', # Till Lindemann
#               '3w4VAlllkAWI6m0AV0Gn6a', # hurts,
#                '4njdEjTnLfcGImKZu1iSrz', # awolnation
#                '3Bd1cgCjtCI32PYvDC3ynO', # london gram
#                '3XHO7cRUPCLOr6jwp8vsx5', # alt j
#                '163tK9Wjr9P9DmM0AVK7lm', # lorde
#                '3iOvXCl6edW5Um0fXEBRXy', # xx
#               '0RqtSIYZmd4fiBKVFqyIqD', # 30 second
#                '5Pwc4xIPtQLFEnJriah9YJ', # one republ
#               '1WgXqy2Dd70QQOU7Ay074N', # aurora,
#                 '4Z8W4fKeB5YxbusRsdQVPb', # radiohead
#                '0BEI7i5sgUuivcfwXLzFmM', # serj tank
#                '3wRt3iJpZDOg73CTUkfv5C', # svrcina
#                 '0YCqg8X5gx0MremW64Ud1h', # ддт
#                 '6mdiAmATAx73kdxrNrnlao', # iron m
#                 '2ye2Wgw4gimLv2eAKyk1NB', # metallica
#                 '1moxjboGR7GNWYIMWsRjgG', # florence + m
#                 '1AZ30JnvQU1pbX6sbRE0Yn', # poets of the fall
#                 '2N0vFuOoMtAQfBmhsRo24e', # kalanda
#                 '4Z8W4fKeB5YxbusRsdQVPb', # radiohead
#                 '1Ffb6ejR6Fe5IamqA5oRUF', # bring me the horizon
#                 '00FQb4jTyendYWaN8pK0wa', # lana del ray
#                 '49qiE8dj4JuNdpYGRPdKbF', # stone sour
#                 '6wWVKhxIU2cEi0K81v7HvP', # rammstein
                '5nGIFgo0shDenQYSE0Sn7c', # Evanescence
                '6XyY86QOPPrYVGvF9ch6wz' # linkin park
]

df_spotify = ht.sp_get_albums_info(client_id, client_secret, artist_list)

In [40]:
# ПОСТ ФИЛЬТРАЦИЯ (АПИ НЕ РАБОТАЕТ ЗДЕСЬ)
ht.sp_get_filter(df_spotify, 
                release_date_min = '2020-01-01', 
                release_date_max = '2030-01-01',
#                 album_flt = ['deluxe', 'live', 'edition', 'soundtrack', 'remix'], # исключаем слова в названии альбомов
                album_flt = [  'edition', 'soundtrack', 'remix'], # исключаем слова в названии альбомов
                album_type_flt = ['single'], # убрать эти типы
                max_artists_in_album = 1,
                other_album_author = False, # false когда не хотим чтобы это был фит артиста в альбом другого чувака
#                  columns_excl = ['uri', 'album_type'] # какие колонки не отображать здесь
                 columns_excl = [] # какие колонки не отображать здесь
                 )

,artist,album,release_date,tracks_cnt,uri,album_artists,album_type
0,Linkin Park,From Zero,2024-11-15,11,spotify:album:4R6FV9NSzhPihHR0h4pI93,['Linkin Park'],album
1,Linkin Park,From Zero: A Cappellas + Instrumentals,2024-11-14,21,spotify:album:7qKEQkRXySP0rhjtXJksfg,['Linkin Park'],album
2,Linkin Park,From Zero: A Cappellas,2024-11-14,11,spotify:album:1x6p39FFIESOq8q8B4oQXN,['Linkin Park'],album
3,Linkin Park,Papercuts: Instrumentals,2024-06-28,20,spotify:album:494aUGKXMH5ruXtR3O1a3H,['Linkin Park'],album
4,Linkin Park,Papercuts,2024-04-12,20,spotify:album:1X472EvsSqH09RyrqbtyXU,['Linkin Park'],album
5,Evanescence,The Bitter Truth,2021-03-26,12,spotify:album:1Z6rvvQMOMZw2DgyF4CQ9t,['Evanescence'],album
